# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [108]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [109]:
# your code here
orders = pd.read_csv('orders.zip')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [127]:
# your code here
# Sub Problem 1
orders_byID = orders.groupby(["CustomerID"]).sum()

In [128]:
# Sub Problem 2
VIP_bool_list = list(orders_byID['amount_spent'] > orders_byID['amount_spent'].quantile(.95))
orders_byID_VIP = orders_byID[VIP_bool_list]

BelowEqual95_list = list(orders_byID['amount_spent'] <= orders_byID['amount_spent'].quantile(.95)) 
Above75_list = list(orders_byID['amount_spent'] > orders_byID['amount_spent'].quantile(.75)) 
pref_bool_list = list(np.logical_and(BelowEqual95_list, Above75_list))

orders_byID_Pref = orders_byID[pref_bool_list]

In [129]:
# Sub Problem 3
Is_VIP_df = orders_byID_VIP.drop([i for i in orders_byID_VIP.columns if i != 'Total'], axis = 1)
Is_VIP_df['Is VIP'] = True

Is_Pref_df = orders_byID_Pref.drop([i for i in orders_byID_Pref.columns if i != 'Total'], axis = 1)
Is_Pref_df['Is Pref'] = True

orders_byID_Status = pd.merge(orders_byID, Is_VIP_df,how='outer', on='CustomerID')
orders_byID_Status['Is VIP'].fillna(value = False, inplace = True)

orders_byID_Status = pd.merge(orders_byID_Status, Is_Pref_df,how='outer', on='CustomerID')
orders_byID_Status['Is Pref'].fillna(value = False, inplace = True)

orders_byID_Status['Is Either'] = orders_byID_Status['Is VIP'] | orders_byID_Status['Is Pref']
orders_byID_Status = orders_byID_Status.drop([i for i in orders_byID_Status.columns if i not in ['CustomerID', 'Is VIP', 'Is Pref', 'Is Either','Country']], axis = 1)


orders_byID_Status

,Is VIP,Is Pref,Is Either
CustomerID,,,
12346,True,False,True
12347,False,True,True
12348,False,True,True
12349,False,True,True
12350,False,False,False
...,...,...,...
18280,False,False,False
18281,False,False,False
18282,False,False,False


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [130]:
# your code here
orders_country = orders.drop([i for i in orders.columns if i not in ['CustomerID', 'Country']], axis = 1)
orders_country = orders_country.drop_duplicates()
orders_country          

orders_byID_Status = pd.merge(orders_byID_Status, orders_country,how='outer', on='CustomerID')
orders_byID_Status.head(5)

,CustomerID,Is VIP,Is Pref,Is Either,Country
0,12346,True,False,True,United Kingdom
1,12347,False,True,True,Iceland
2,12348,False,True,True,Finland
3,12349,False,True,True,Italy
4,12350,False,False,False,Norway


In [131]:
orders_byID_Status_Counters = orders_byID_Status.groupby(["Country"]).sum()
orders_byID_Status_Counters[orders_byID_Status_Counters['Is VIP'] == orders_byID_Status_Counters['Is VIP'].max()]

,CustomerID,Is VIP,Is Pref,Is Either
Country,,,,
United Kingdom,61016530,177,755,932


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [132]:
# your code here
orders_byID_Status_Counters[orders_byID_Status_Counters['Is Either'] == orders_byID_Status_Counters['Is Either'].max()]

,CustomerID,Is VIP,Is Pref,Is Either
Country,,,,
United Kingdom,61016530,177,755,932
